# Tarea 10 - Sebastián Negrete Aragón - Física Computacional
# Semestre 2017-2

Con este programa haremos un análisis similar al hecho por el profesor con el código para la derivación hacia adelante pero para la derivación simétrica

In [1]:
#Usaremos SymPy
using SymPy

Calcularemos los coeficientes de Taylor, en el código del notebook de la clase se usa $log(x+1)$, aquí será con $arcsinh(x)$ el cuál en _SymPy_ es $asinh(x)$.

In [12]:
#Función que nos regresa un Float de Julia
function coeff_Taylor_asinh(i)
    x=Sym("x")
    h=Sym("h")
    N(subs(diff(asinh(x),x,i)/factorial(Float64(i)),x=>0))
end

In [13]:
#Función que nos regresa una expresión de SymPy
function coeff_Taylor_asinh_rational(i)
    x=Sym("x")
    h=Sym("h")
    subs(diff(asinh(x),x,i)/factorial(i),x=>0)
end

Ahora damos la función `finite_difference_symmetric` que representa las diferencias finitas simétricas:

$\delta_h:f(x)\mapsto f(x+h)-f(x-h)$

In [24]:
#Función diferencia finita simétrica
function finite_difference_symmetric(f)
    return (x,h)->f(x+h,h)-f(x-h,h) #La notación indica que a (x,h) lo evaluará en lo que va después de la flecha ->
end;

La justificación del porqué `finite_difference_forward` es una función de dos argumentos es análoga al código expuesto en el notebook de la clase: permite imponer orden en las variables y facilitar la concatenación del operador.
Ahora viene el código que nos permitirá concatenar. Así el _output_ tiene la misma forma que el _input_, algo que nos permitirá retroalimentar la función para así concatenar

In [25]:
#Función g definida como función de SymPy
g=SymFunction("g")

In [26]:
#Definimos la función rr, con esto la función de una variable g(x) ahora tiene dependencia (x,h)
rr(x,h)=g(x);

In [27]:
#La función gg la definimos como `finite_difference_symmetric` aplicado a rr
gg=finite_difference_symmetric(rr);

In [28]:
#Con esto arrojamos la expresión resultante de gg
gg(Sym("x"),Sym("h"))

-g(-h + x) + g(h + x)

Con esto hemos concatenado satisfactoriamente el operador de diferencia finita simétrica. Ahora haciendo la segunda concatenación de `finite_difference_symmetric`:

In [19]:
#Segunda concatenación
gg=finite_difference_symmetric(gg);
gg(Sym("x"),Sym("h"))

-2*g(x) + g(-2*h + x) + g(2*h + x)

Con esto estamos listos para dar las derivaciones con errores de orden superior, para esto recordemos:
$$\:\left.\frac{d}{d\xi}\right|_{\substack{\xi=x}}=\frac{1}{2h}\sum_{n=0}^{\infty}\frac{(-1)^{n}(2n)!}{4^{2n}(n!)^{2}(2n+1)}\delta_{h}^{\:2n+1}=\frac{1}{2h}\left(\delta_{h}-\frac{1}{24}\delta_{h}^{\:2}+\cdots\right)\ \ldots\ Obs.\ 1$$
Damos dos expresiones para dicha derivación debido a que tenemos dos funciones para los coeficientes de Taylor:

In [31]:
#Esta función nos da la derivada numérica simétrica como Float de Julia
#Usamos la función de coeficientes de Taylor que arroja Float de Julia
#f: función
#x: punto a aproximar
#h: incremento
#order: orden de la aproximación
function Derivative_order(f,x,h,order)
    g(x,h)=f(x)
    der=0
    for i in range(1,order)
        g=finite_difference_symmetric(g)
        der+=coeff_Taylor_asinh(i)*g(x,h)/2^i   #Obs. 1 y se divide entre i por tener la función asinh(x)
    end
    der/h
end

Derivative_order (generic function with 1 method)

In [30]:
#Esta función nos da la derivada simétrica como expresión de SymPy
#Usamos la función de coeficientes de Taylor que la expresión de SymPy
#f: función
#x: punto a aproximar
#h: incremento
#order: orden de la aproximación
function Derivative_order_rational_coefficients(f,x,h,order)
    g(x,h)=f(x)
    der=0
    for i in range(1,order)
        g=finite_difference_symmetric(g)
        der+=coeff_Taylor_asinh_rational(i)*g(x,h)/2^i #Obs. 1 y se divide entre i por tener la función asinh(x)
    end
    der/h
end

Derivative_order_rational_coefficients (generic function with 1 method)

__Prueba__ Con la función que arroja float de Julia

In [21]:
#Aplicamos para la función cos(x) en el punto 10 con un incremento 0.01 y un orden 5
Derivative_order(cos,10.0,0.01,5)

0.5440211108893371

In [22]:
#La derivada de cos(x) es -sin(x), el valor de la derivada de cos(x) es:
-sin(10.0)

0.5440211108893698

Ahora para conocer como escala el error para comparar con la derivada hacia adelante usamos la función que arroja expresiones en SymPy

In [32]:
#Expresión de la derivada para un orden 5
approx=Derivative_order_rational_coefficients(SymFunction("f"),Sym("x"),Sym("h"),5)

  3*f(-5*h + x)   25*f(-3*h + x)   75*f(-h + x)   75*f(h + x)   25*f(3*h + x) 
- ------------- + -------------- - ------------ + ----------- - ------------- 
       1280            768             128            128            768      
------------------------------------------------------------------------------
                                             h                                

  3*f(5*h + x)
+ ------------
      1280    
--------------
              

In [33]:
#Expandimos en serie de Taylor:
series(approx,Sym("h"),0,8)

                               /   7           \|              
                             6 |  d            ||              
                          5*h *|------(f(xi_1))||              
                               |     7         ||              
/  d           \|              \dxi_1          /|xi_1=x    / 8\
|-----(f(xi_1))||       + ----------------------------- + O\h /
\dxi_1         /|xi_1=x                112                     

Para la derivada simétrica de orden 5 se tiene que el error escala como $h^{6}$. Se mejoró la aproximación respecto a la derivada hacia adelante propuesta en clase.

__Fin__